In [ ]:
# En caso de trabajar en otra plataforma que no sea Databricks
# instalar la librería PySpark
# pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import * 

In [ ]:
# Iniciar Spark en otra plataforma que no sea Databricks
# spark = SparkSession.builder.appName('DataFrame').getOrCreate()

Conexión a Data Lake y descarga de tablas

In [ ]:
container = '' # Nombre del contenedor en Azure Data Lake
datalake = '' # Nombre de la instancia de Azure Data Lake (o Storage Account)

In [ ]:
access_key = '*******' # Clave de acceso a Azure Data Lake
spark.conf.set(f'fs.azure.account.key.{datalake}.dfs.core.windows.net', access_key)

In [ ]:
file_name = ['Company_Tweet', 'Company', 'CompanyValues', 'Tweet']

In [ ]:
for name in file_name:
    exec(f"PATH_{name} = f'abfss://{container}@{datalake}.dfs.core.windows.net/Trusted/{name}/*'")
    exec(f"df_{name} = spark.read.load(PATH_{name}, format = 'parquet',inferSchema = True)")

In [ ]:
# Creación de vista temporal para realizar consultas SQL
for name in file_name:
    exec(f"df_{name}.createOrReplaceTempView('df_{name}')")

Join de tabla Tweets con tabla de compañias y nombres

In [ ]:
df_Tweets = spark.sql('''
          SELECT Tweet.tweet_id,
                Tweet.writer,
                Tweet.post_date,
                CAST(Tweet.post_date AS DATE) AS date,
                CTweet.ticker_symbol,
                df_Company.company_name,
                Tweet.body,
                Tweet.comment_num,
                Tweet.retweet_num,
                Tweet.like_num,
                `Tweet`.`open_market`
                
          FROM df_Tweet AS Tweet
          LEFT JOIN df_Company_Tweet AS CTweet ON Tweet.tweet_id = CTweet.tweet_id
          LEFT JOIN df_Company ON CTweet.ticker_symbol = df_Company.ticker_symbol
          ''')

In [ ]:
df_Tweets.groupBy(col('ticker_symbol')).count().show()

+-------------+-------+
|ticker_symbol|  count|
+-------------+-------+
|         AAPL|1415653|
|         TSLA|1071279|
|         GOOG| 390538|
|        GOOGL| 325561|
|         AMZN| 713038|
|         MSFT| 373689|
+-------------+-------+



In [ ]:
df_Tweets.createOrReplaceTempView('df_Tweets')

Calculos de cantidades diarias de la tabla Tweets

In [ ]:
Tweets_diarios = spark.sql('''
          SELECT t.date,
          t.interaction_open,
          t.interaction_close,
          t.ticker_symbol,
          t.interaction_open + interaction_close AS interaction_daily,
          t.tweets_daily,
          t.comment_num,
          t.retweet_num,
          t.like_num
          FROM (
              SELECT date, ticker_symbol,
                     SUM(CASE WHEN `open_market` = TRUE 
                              THEN comment_num + retweet_num + like_num 
                              ELSE 0 
                     END) AS interaction_open,
                     SUM(CASE WHEN `open_market` = FALSE 
                              THEN comment_num + retweet_num + like_num 
                              ELSE 0 
                     END) AS interaction_close,
                     count(ticker_symbol) AS tweets_daily,
                     sum(comment_num) AS comment_num,
                     sum(retweet_num) AS retweet_num,
                     sum(like_num) AS like_num
              FROM df_Tweets
              GROUP BY date, ticker_symbol
          ) AS t
          ORDER BY t.date, ticker_symbol
          ''')

In [ ]:
Tweets_diarios.createOrReplaceTempView('df_TweetsDiarios')

Joins de tabla de CompanyValues con la tabla creada anteriormente de Tweets diarios

In [ ]:
CompanyValues = spark.sql('''
                    SELECT CV.ticker_symbol,
                    C.company_name,
                    CV.day_date,
                    CV.open_value,
                    CV.close_value,
                    CV.volume,
                    CV.high_value,
                    CV.low_value,
                    CV.v_volume,
                    CV.v_open_diaria,
                    CV.v_open,
                    CV.mg_open_volume,
                    TD.interaction_open,
                    TD.interaction_close,
                    TD.interaction_daily,
                    TD.tweets_daily,
                    TD.comment_num,
                    TD.retweet_num,
                    TD.like_num
                    FROM df_CompanyValues AS CV
                    LEFT JOIN df_Company AS C ON CV.ticker_symbol = C.ticker_symbol
                    LEFT JOIN df_TweetsDiarios AS TD ON CV.ticker_symbol = TD.ticker_symbol AND CV.day_date = TD.date
                    ORDER BY CV.ticker_symbol, CV.day_date
                    ''')

In [ ]:
CompanyValues.createOrReplaceTempView('df_CompanyValues')

Calculos de variaciones

In [ ]:
CompanyValues = spark.sql('''
                        SELECT *
                        FROM (
                            SELECT *, interaction_daily - lag(interaction_daily,1) OVER (PARTITION BY ticker_symbol ORDER BY day_date) AS v_interaccion,
                            v_interaccion / v_volume as mg_interaccion_volume,
                            tweets_daily - lag(tweets_daily,1) OVER (PARTITION BY ticker_symbol ORDER BY day_date) AS v_tweets,
                            (v_interaccion + v_tweets) / v_volume AS mg_interacion_tweets_volumen
                            FROM df_CompanyValues
                            )
                        ''')

In [ ]:
CompanyValues.createOrReplaceTempView('df_CompanyValues')

In [ ]:
CompanyValues = spark.sql('''
                             SELECT ticker_symbol,
                             company_name,
                             day_date,
                             open_value,
                             close_value,
                             volume,
                             high_value,
                             low_value,
                             coalesce(v_volume, 0) AS v_volume,
                             coalesce(v_open_diaria, 0) AS v_open_diaria,
                             coalesce(v_open, 0) AS v_open,
                             coalesce(mg_open_volume, 0) AS mg_open_volume,
                             coalesce(interaction_open, 0) AS interaction_open,
                             coalesce(interaction_close, 0) AS interaction_close,
                             coalesce(interaction_daily, 0) AS interaction_daily,
                             coalesce(tweets_daily, 0) AS tweets_daily,
                             coalesce(v_interaccion, 0) AS v_interaccion,
                             coalesce(mg_interaccion_volume, 0) AS mg_interaccion_volume,
                             coalesce(mg_interacion_tweets_volumen, 0) AS mg_interacion_tweets_volumen,
                             coalesce(v_tweets, 0) AS v_tweets,
                             coalesce(comment_num, 0) AS comment_num,
                             coalesce(retweet_num, 0) AS retweet_num,
                             coalesce(like_num, 0) AS like_num
                             FROM df_CompanyValues
                             ''')

Conexión y carga en base de datos

In [ ]:
jdbcHostname = "" # Nombre o URL de servidor de base de datos
jdbcPort = 1433
jdbcDatabase = "DWIntegrador" # Nombre de la base de datos
jdbcUsername = "" # Nombre de usuario para conexión de base de datos
jdbcPassword = "" # Contraseña del usuario
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcUrl= f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}"

In [ ]:
CompanyValues.write.mode("overwrite")\
.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("dbtable", "Maidana_Financiero")\
.save()